# Table of Contents
1. [Introduction](#intro)
2. [Installing required packages](#install)
3. [Loading  Datasets and Data Description](#data)
    * 3.1.[Fact Market Demand Data](#fact)
    * 3.2.[Customer Demographics Data](#demograpic)
    * 3.3.[Zip Code Data](#zip)
    * 3.4 [Missing Data](#missing)
4. [Modeling](#modeling)  
    * 4.1.[Linear Regression](#lr)
        - 4.1.1 [Encoding](#encode)
    * 4.1.[Autoregression (AR) Model](#ar)
    * 4.2.[ARIMA Model](#arima)
    * 5.1.[Prophet Model](#prophet)
        - 5.1.1 [Holiday Data](#holiday)
7. [Conclusion](#Conclusion)
8. [Group Member Contribution](#group)

# <a name="intro"></a> 1. Introduction

Swire Coca-Cola  is continually introducing innovative products into the market. Innovation diversifies product offerings, meeting evolving consumer trends. This strategy, marked by unique products and premium pricing, expands market reach and attracts new customers. It responds to technology changes and regulations, ensuring cost efficiency and compliance. Additionally, it rejuvenates sales and marketing. In essence, product innovation is essential for staying relevant, driving revenue, and sustaining long-term growth in a dynamic market.

So, Swire Coca-Cola wants to optimize its production planning and inventory management for these novel beverages. The challenge for Swire is to forecast the demand for these new products accurately to ensure optimal production quantities, prevent out-of-stock and overproduction, minimize costs, and maximize customer satisfaction. The Business problem is to accurately predict the weekly demand of the innovation products. The focus of our project will be on optimal production planning, cost minimization, and market location and date prediction.



The Analytical Problem is to forecast the details expected by the Swire for the 7 innovation products that are planned to be launched in the near future. The basic details of each of the new innovation products are provided so that it can be compared against the historical data and can be used to forecast the sales of the innovation products as close as possible to the real time sales.

The key factors for the Items are the Caloric Segment, Market Category, Manufacturer, Brand, Package Type and Flavor which will be used as the independent variables from the Previous years data to create a Forecast for the new innovation Products. So, the Target variable is the Dollar Sales/unit sales for the innovation products.


In this notebook we will create the forecasting models to predict the demand for the new innovation products that the Swire team is expecting to launch in the next year. The scope includes creating the end-to-end development of the forecasting models to find the pattern of the sales for the products and also to discover some insights for improving the sales of innovation products.  It aims to provide Swire with the insights and tools necessary to maintain its Sales while making data-driven decisions that can contribute to the success of the innovation Products.

# <a name="install"></a>2. Installing required packages

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
!pip install google.cloud.bigquery

In [27]:
import os
import numpy as np
import pandas as pd
import math
import itertools
from scipy import stats
import time


#matplotlib libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mp
import matplotlib.image as mpimg
import matplotlib.cm as cm
import matplotlib.colors
import seaborn as sns

#date libraries
from dateutil import parser
from datetime import datetime, timedelta, date
import holidays

#prophet library
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

#xgboost
from xgboost import XGBRegressor

#sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


In [28]:

#pandas options
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#matplotlib setting defaults
sns.set(
        rc={
 "axes.axisbelow": False,
 "axes.edgecolor": "lightgrey",
 "axes.facecolor": "None",
 "axes.grid": False,
 "axes.labelcolor": "dimgrey",
 "axes.spines.right": False,
 "axes.spines.top": False,
 "figure.facecolor": "white",
 "lines.solid_capstyle": "round",
 "patch.edgecolor": "w",
 "patch.force_edgecolor": True,
 "text.color": "dimgrey",
 "xtick.bottom": False,
 "xtick.color": "dimgrey",
 "xtick.direction": "out",
 "xtick.top": False,
 "ytick.color": "dimgrey",
 "ytick.direction": "out",
 "ytick.left": False,
 "ytick.right": False})


In [29]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

from google.colab import auth

# Authenticate user
auth.authenticate_user()

In [30]:
# Create a BigQuery client
client = bigquery.Client(project='is-6813-spring-24')

# <a name="data"></a> 3. Loading  Datasets and Data Description

## <a name = "fact"> </a> 3.1 Fact Market Demand Data

The main data file is the fact_market_demand data since that seems to hold the most value for answering the business problem. There are 10 fields and 24,461,424 rows. The columns are data, unit_sales, caloric_segment, category, manufacturer, brand, package, item, market_key and dollar_sales.

The date range is nearly three full years, spanning December 2020 to October 2023. The caloric segment is a categorical column with values Regular and Diet/light.


In [31]:
# Get Fact_Market_Demand Data from BigQuery

fact_demand_query = """
select * from `swire.fact_market_demand`
"""
# Execute the query
fact_demand_query_job = client.query(fact_demand_query)

# Get the results as a Pandas DataFrame
Fact_Demand_data = fact_demand_query_job.to_dataframe()

# Display Fact_market_demand data
Fact_Demand_data.head()

,DATE,MARKET_KEY,CALORIC_SEGMENT,CATEGORY,UNIT_SALES,DOLLAR_SALES,MANUFACTURER,BRAND,PACKAGE,ITEM
0,2022-04-02,464,DIET/LIGHT,SSD,35.00,194.35,COCOS,DIET BUBBLE JOY ADVANTAGEOUS CF,12SMALL 8ONE SHADYES JUG,DIET BUBBLE JOY ADVANTAGEOUS CAFFEINE FREE GEN...
1,2023-04-22,951,DIET/LIGHT,SSD,67.00,124.61,COCOS,DIET BUBBLE JOY ADVANTAGEOUS CF,2L MULTI JUG,DIET BUBBLE JOY ADVANTAGEOUS CAFFEINE FREE GEN...
2,2021-01-09,882,REGULAR,SSD,14.00,25.37,COCOS,CINNAMON BUBBLE JOY ADVANTAGEOUS,20SMALL MULTI JUG,KOOL! GENTLE DRINK CINNAMON COLA JUG 20 LIQUID...
3,2022-10-22,951,DIET/LIGHT,SSD,6.00,9.00,SWIRE-CC,DIET SPARKLING JACCEPTABLETLESTER,1L MULTI JUG,DIET SPARKLING JACCEPTABLETLESTER TONIC WATER ...
4,2021-10-30,613,DIET/LIGHT,SSD,94.00,490.49,JOLLYS,HILL MOISTURE ZERO SUGAR MAJOR MELON,12SMALL 12ONE CUP,RAINING ZERO SUGAR GENTLE DRINK MAJOR CANES A...


In [32]:
# Get a summary of the dataset
print(Fact_Demand_data.describe())

       MARKET_KEY  UNIT_SALES  DOLLAR_SALES
count 24461424.00 24461424.00   24461424.00
mean       593.14      174.37        591.14
std        605.88      857.81       3040.54
min          1.00        0.04          0.01
25%        260.00       11.00         36.59
50%        547.00       40.00        135.05
75%        845.00      126.00        427.14
max       6802.00    96776.00     492591.07


In [33]:
print(Fact_Demand_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24461424 entries, 0 to 24461423
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DATE             dbdate 
 1   MARKET_KEY       Int64  
 2   CALORIC_SEGMENT  object 
 3   CATEGORY         object 
 4   UNIT_SALES       float64
 5   DOLLAR_SALES     float64
 6   MANUFACTURER     object 
 7   BRAND            object 
 8   PACKAGE          object 
 9   ITEM             object 
dtypes: Int64(1), dbdate(1), float64(2), object(6)
memory usage: 1.8+ GB
None


In [34]:
# Check for missing values
print(Fact_Demand_data.isnull().sum())

DATE                   0
MARKET_KEY             0
CALORIC_SEGMENT    59725
CATEGORY               0
UNIT_SALES             0
DOLLAR_SALES           0
MANUFACTURER           0
BRAND                  0
PACKAGE                0
ITEM                   0
dtype: int64




In the Fact Demand data we can see that there are 59725 missing data in the CALORIC_SEGMENT Column. Before replacing that let us look at how the CALORIC_SEGMENT data is disctributed by using the count SQL query from Big Query as the processing time will be more if done from dataframe object.



In [35]:
# Get Caloric Segment data from Fact_Market_Demand Data from BigQuery

CS_query = """
select CALORIC_SEGMENT,
count(*) as Count_caloric_segment
from `swire.fact_market_demand`
group by CALORIC_SEGMENT;
"""
# Execute the query
CS_job = client.query(CS_query)

# Get the results as a Pandas DataFrame
CS_data = CS_job.to_dataframe()

CS_data.head()

,CALORIC_SEGMENT,Count_caloric_segment
0,REGULAR,12231585
1,DIET/LIGHT,12170114
2,None,59725


## <a name = "demographic"> </a> 3.2 Customer Demographics Data

The customer_demographics data provides information on the demographics with respect to zip codes like Age, Gender with age, Education, Income range, Marraige status, Race and Household information.

All of these demographic information is combined and can be divided by filtering the 'Segment' field for each type like Household income, education, etc.

In [36]:
# Get consumer_demographics Data from BigQuery

demograph_query = """
select * from `swire.consumer_demographics`
"""
# Execute the query
demograph_query_job = client.query(demograph_query)

# Get the results as a Pandas DataFrame
Demographics_data = demograph_query_job.to_dataframe()

# Display Fact_market_demand data
Demographics_data.head()

,Geography_Name,Zip,City,State,Segment,Criteria,Count,Criteria_Unit
0,"57064 Tea, SD",57064,Tea,SD,Household Income,"Income Under $10,000",36,HHs
1,"57064 Tea, SD",57064,Tea,SD,Household Income,"Income $10,000 - $19,999",86,HHs
2,"57064 Tea, SD",57064,Tea,SD,Household Income,"Income $20,000 - $29,999",87,HHs
3,"57064 Tea, SD",57064,Tea,SD,Household Income,"Income $30,000 - $39,999",137,HHs
4,"57064 Tea, SD",57064,Tea,SD,Household Income,"Income $40,000 - $49,999",85,HHs


In [37]:
# Check for missing value in Demographics data
print(Demographics_data.isnull().sum())

Geography_Name    0
Zip               0
City              0
State             0
Segment           0
Criteria          0
Count             0
Criteria_Unit     0
dtype: int64


## <a name = "zip"> </a> 3.3 Zip Code Data

This zip to market unit mapping data contains only Zip code to the market key information. This information can be used to connect the Demographics data to the Fact_Market_Demand data and also to know which Market key belongs to which region in country based on the Zip codes.

In [38]:
# Get zip_to_market_unit_mapping Data from BigQuery

zip_market_query = """
select * from `swire.zip_to_market_unit_mapping`
"""
# Execute the query
zip_market_query_job = client.query(zip_market_query)

# Get the results as a Pandas DataFrame
zip_market_data = zip_market_query_job.to_dataframe()

# Display Fact_market_demand data
zip_market_data.head()

,ZIP_CODE,MARKET_KEY
0,83462,1
1,83463,1
2,83466,1
3,83467,1
4,83525,1


In [39]:
# Check for missing values in Zip Market data
print(zip_market_data.isnull().sum())

ZIP_CODE      0
MARKET_KEY    0
dtype: int64


## 3.5.<a name="features"></a> Feature Engineering

Before starting the Data Exploration, we want to include few of the new columns derived from the original data and added into the data.

Let us join the Demographics data with the Zip_to_market Data to get the States for each of the Market keys. Since some Market_keys contain zipcodes of multiple states we divide states by '/' and keep it in one column named 'States'.

Then we will attach the States column to the main fact_demand_data so that we get the states information for each of sales data.

We will get all this from Bigquery to reduce the run time.

In [ ]:
#Google Big Query
ZMS_query = """
SELECT DISTINCT c.*, d.States
FROM `is-6813-spring-24.swire.fact_market_demand` as c
inner join
( SELECT a.MARKET_KEY, STRING_AGG(DISTINCT b.State, '/') AS States
FROM `is-6813-spring-24.swire.zip_to_market_unit_mapping` AS a
INNER JOIN `is-6813-spring-24.swire.consumer_demographics` AS b
ON b.Zip = a.ZIP_CODE
GROUP BY a.MARKET_KEY )
as d
ON c.MARKET_KEY = d.MARKET_KEY
ORDER BY c.DATE, c. MARKET_KEY, c.ITEM
"""
# Execute the query
ZMS_job = client.query(ZMS_query)

# Get the results as a Pandas DataFrame
fact_demand_state_data = ZMS_job.to_dataframe()

fact_demand_state_data.head()

In [ ]:
# Convert DATE column to datetime
fact_demand_state_data['DATE'] = pd.to_datetime(fact_demand_state_data['DATE'])


In [ ]:
fact_demand_state_data.head()

##<a name="missing"></a> 3.4. Missing Data

Since both the Regular and Diet/light categories are equal we are unable to categorize the null value to any of the categories without disturbing the other data present. So, We can just replace that Null values with the string of 'Unknown', so that the data is not impacted.

In [ ]:
# Handling missing values

fact_demand_state_data['CALORIC_SEGMENT'].fillna('Unknown', inplace=True)

In [ ]:
# checking for missing data again

print(fact_demand_state_data.isnull().sum())

In [ ]:
# Handling missing values

Fact_Demand_data['CALORIC_SEGMENT'].fillna('Unknown', inplace=True)

In [22]:
# checking for missing data again

print(Fact_Demand_data.isnull().sum())

DATE               0
MARKET_KEY         0
CALORIC_SEGMENT    0
CATEGORY           0
UNIT_SALES         0
DOLLAR_SALES       0
MANUFACTURER       0
BRAND              0
PACKAGE            0
ITEM               0
dtype: int64


# 4. <a name="modeling"></a> Modeling

First defining the required functions like missing_data and MAPE which will help us further in the Modeling Process and for finding the best model for Forecasting.

In [23]:
def missing_data(input_data):
    '''
    This function returns dataframe with information about the percentage of nulls in each column and the column data type.

    input: pandas df
    output: pandas df

    '''

    total = input_data.isnull().sum()
    percent = (input_data.isnull().sum()/input_data.isnull().count()*100)
    table = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    types = []
    for col in input_data.columns:
        dtype = str(input_data[col].dtype)
        types.append(dtype)
    table["Types"] = types
    return(pd.DataFrame(table))

def mape(actual, pred):
    '''
    Mean Absolute Percentage Error (MAPE) Function

    input: list/series for actual values and predicted values
    output: mape value
    '''
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


## <a name = "prophet"> </a> 5.1. Prophet Model

## <a name = "holiday"> </a> 5.1.1. Holiday Data

First getting reading the holiday data for the US so that the pattern for the holidays can be understood. The lower window is for -2 so that it can consider 2 dys before the holiday and 1 for upper window, so that it can consider one day after the holiday

In [ ]:
#adding holiday data

holiday = pd.DataFrame([])

for date_, name in sorted(holidays.US(years=[2020,2021,2022,2023,2024]).items()):
    holiday = pd.concat([holiday, pd.DataFrame({'ds': date_, 'holiday': "US-Holidays", 'lower_window': -2, 'upper_window': 1}, index=[0])], ignore_index=True)

holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')
holiday.head()

In [ ]:
#converting the column names to lower characters and replacing spaces by underscore
fact_demand_state_data.columns = fact_demand_state_data.columns.str.replace(' ', '_').str.lower()

#converting the date format to year-month-date
fact_demand_state_data['date'] = pd.to_datetime(fact_demand_state_data['date'], format= "%Y/%m/%d")

In [ ]:
# Finding the minimum and maximum dates in the data
min(fact_demand_state_data['date']), max(fact_demand_state_data['date'])

In [ ]:
fact_demand_state_data.head()

## <a name = "Q1"> </a> 5.1.2. Question 1

Writing the segments for the Qestion 1.  
a. caloric Segment - Diet  
b. Market Category - SSD   
c. Manufacturer - Swire-CC  
d. Brand - Diet Smash  
e. Package Type - 11Small4One  
f. Flavor - Plum

Now, checking which lines in the data will yield to the most closest for our forecast.





In [ ]:

q1_f1 = fact_demand_state_data['caloric_segment'] == 'DIET/LIGHT'
q1_f2 = fact_demand_state_data['category'] == 'SSD'
q1_f3 = fact_demand_state_data['brand'] == 'DIET SMASH'
q1_f4 = fact_demand_state_data['item'].str.lower().str.contains('plum')

filter_condition_q1 = (( q1_f1 & q1_f2 & q1_f3 ) | ( q1_f4 & q1_f2 & q1_f1 ))

q1_df = fact_demand_state_data[filter_condition_q1]

q1_df.head()

In [ ]:
q1_df.shape

In [ ]:
min(q1_df['date']), max(q1_df['date'])

In [ ]:
q1_agg_df = q1_df.groupby(['date']).agg({'unit_sales': 'sum', 'item': 'nunique'}).reset_index().sort_values(['date'])
q1_agg_df['unit_sales'] = q1_agg_df['unit_sales'] / q1_agg_df['item']
q1_agg_df.drop(columns=['item'],inplace=True)


In [ ]:
#q1_agg_df = q1_df.groupby(['date']).agg({'unit_sales':'sum'}).reset_index().sort_values(['date'])

In [ ]:
q1_agg_df.head()

In [ ]:
# @title date vs sum of unit_sales

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
    # Aggregate sum of unit sales by date
  summed = series.groupby('date')['unit_sales'].sum().reset_index()

  xs = summed['date']
  ys = summed['unit_sales']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

# Example usage:
fig, ax = plt.subplots(figsize=(10, 5.2))
df_sorted = q1_agg_df.sort_values('date', ascending=True)
_plot_series(df_sorted, 'Sum of Unit Sales')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
plt.ylabel('Sum of Unit Sales')
plt.legend()
plt.show()

In [ ]:
missing_data(q1_agg_df).head()

In [ ]:
#total_sales_df1 = q1_agg_df.dropna()

In [ ]:
q1_agg_df.shape

In [ ]:
Q1_sales_data_df = q1_agg_df[q1_agg_df.date<='2023-06-05']
Q1_test_set_df = q1_agg_df[q1_agg_df.date>='2023-06-05']

In [ ]:
Q1_sales_data_df.shape

In [ ]:
Q1_test_set_df.shape

In [ ]:
changepoint_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()
print(changepoint_prior_scale_range)

In [ ]:
seasonality_prior_scale_range = np.linspace(0.01, 10, num=5).tolist()


In [ ]:
start_time = time.time()

#category_df = Q1_sales_data_df[['date','unit_sales']]

def best_params(category_df):
    category_df.rename(columns={'date': 'ds', 'unit_sales': 'y'}, inplace=True)
    category_df[["y"]] = category_df[["y"]].apply(pd.to_numeric)
    category_df["ds"] = pd.to_datetime(category_df["ds"])

    param_grid = {
        "changepoint_prior_scale": changepoint_prior_scale_range,
        "seasonality_prior_scale": seasonality_prior_scale_range }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    mapes = []

    # Use cross validation to evaluate all parameters
    for params in all_params:
         m = Prophet(**params).fit(category_df)  # Fit model with given params
         df_cv = cross_validation(m, initial="365 days", period="30 days", horizon = "30 days") #took parallel out
         df_p = performance_metrics(df_cv, rolling_window=1)
         mapes.append(df_p["mape"].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results["mape"] = mapes

    params_dict = dict(tuning_results.sort_values("mape").reset_index(drop=True).iloc[0])

    return params_dict

params1_dict = best_params(Q1_sales_data_df)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
params1_dict


In [ ]:
prediction_days = 16
forecast_start_date = max(Q1_sales_data_df.ds) - timedelta(prediction_days)

In [ ]:
#PROPHET MODEL

#model
m = Prophet(changepoint_prior_scale=params1_dict['changepoint_prior_scale'],
            seasonality_prior_scale=params1_dict['seasonality_prior_scale'],
            holidays=holiday)

m.fit(Q1_sales_data_df)

future = m.make_future_dataframe(periods=52, freq='W-SAT')
fcst_prophet_train = m.predict(future)

filter = fcst_prophet_train['ds']>=forecast_start_date
predicted_df = fcst_prophet_train[filter][['ds','yhat']]
predicted_df = predicted_df.merge(Q1_sales_data_df)

print(mape(predicted_df['y'],predicted_df['yhat']))

In [ ]:
print(forecast_start_date)

In [ ]:
fig1 = m.plot(fcst_prophet_train)
fig2 = m.plot_components(fcst_prophet_train)

forecasted_df = fcst_prophet_train[fcst_prophet_train['ds']>=forecast_start_date]


In [ ]:
forecast_start_date = '2023-06-05'

future = m.make_future_dataframe(periods=16, freq='W-SAT')
fcst_prophet_train = m.predict(future)

filter = fcst_prophet_train['ds']>=forecast_start_date
predicted_df = fcst_prophet_train[filter][['ds','yhat']]



# Calculate Mean Absolute Percentage Error (MAPE)
mape_v = mape(Q1_test_set_df['unit_sales'],predicted_df['yhat'])


# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(Q1_test_set_df['unit_sales'],predicted_df['yhat'])

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(Q1_test_set_df['unit_sales'],predicted_df['yhat'])

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(Q1_test_set_df['unit_sales'],predicted_df['yhat'])

print(f"-----------------------Features---------------------------")

print(f'Mean Absolute Error- MAE: {mae}')
print(f'Mean Squared Error- MSE: {mse}')
print(f'Root Mean Square Error- RMSE: {rmse}')
print(f'R2:R-squared{r2}')
print(f'Mean Absolute Percentage Error- MAPE: {mape_v}')

In [ ]:
forecast_start_date = '2023-12-30'

future = m.make_future_dataframe(periods=85, freq='W-SAT')
fcst_prophet_train = m.predict(future)

filter = fcst_prophet_train['ds']>=forecast_start_date
predicted_13_df = fcst_prophet_train[filter][['ds','yhat']]

predicted_13_df.tail()

In [ ]:
# Initialize variables to store the maximum sum and its corresponding index
max_sum = float('-inf')
max_index = -1

# Calculate the sum of the first 13 values as the initial maximum sum
current_sum = predicted_13_df['yhat'].iloc[:13].sum()

# Iterate through the DataFrame to find the maximum sum
for i in range(len(predicted_13_df) - 13):
    if current_sum > max_sum:
        max_sum = current_sum
        max_index = i

    # Update the sum for the next window
    current_sum = current_sum - predicted_13_df['yhat'].iloc[i] + predicted_13_df['yhat'].iloc[i + 13]

# Print the maximum sum and its corresponding index
print("Maximum sum:", max_sum)
print("Index of the first element in the set:", max_index)

print("Date of the start of the 13th year:", predicted_13_df['ds'].iloc[max_index])

print(predicted_13_df.iloc[max_index:max_index+13])


From the above Forecast and the calculations the 13 weeks that would perform the best in the market is the week from 19th October,2024 to 18th January, 2025. The total forecast in demand in unit sales for those weeks is estimated to be 117786.38

## <a name = "Q2"> </a> 5.3. Question 2

Writing the segments for the Qestion 2.  
a. caloric Segment - Regular  
b. Market Category - SSD   
c. Manufacturer - Swire-CC  
d. Brand - Sparkling Jacceptabletlester  
e. Package Type - 11Small MLT  
f. Flavor - Avocado

In [ ]:
q2_f1 = fact_demand_state_data['caloric_segment'] == 'REGULAR'
q2_f2 = fact_demand_state_data['category'] == 'SSD'
q2_f3 = fact_demand_state_data['manufacturer'].str.lower().str.contains('swire')
q2_f4 = fact_demand_state_data['item'].str.lower().str.contains('avocado')

filter_condition_q2 = ( q2_f1 & q2_f2 & q2_f3 & q2_f4 )

q2_df = fact_demand_state_data[filter_condition_q2]

q2_df.head()

In [ ]:
q2_df.shape

In [ ]:
min(q2_df['date']), max(q2_df['date'])

In [ ]:
#q2_agg_df = q2_df.groupby(['date']).agg({'unit_sales':'sum'}).reset_index().sort_values(['date'])

q2_agg_df = q2_df.groupby(['date']).agg({'unit_sales': 'sum', 'item': 'nunique'}).reset_index().sort_values(['date'])
q2_agg_df['unit_sales'] = q2_agg_df['unit_sales'] / q2_agg_df['item']
q2_agg_df.drop(columns=['item'],inplace=True)


In [ ]:
q2_agg_df.head()

In [ ]:
# @title date vs sum of unit_sales for Q2

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
    # Aggregate sum of unit sales by date
  summed = series.groupby('date')['unit_sales'].sum().reset_index()

  xs = summed['date']
  ys = summed['unit_sales']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

# Example usage:
fig, ax = plt.subplots(figsize=(10, 5.2))
df_sorted = q2_agg_df.sort_values('date', ascending=True)
_plot_series(df_sorted, 'Sum of Unit Sales')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
plt.ylabel('Sum of Unit Sales')
plt.legend()
plt.show()

In [ ]:
Q2_sales_data_df = q2_agg_df[q2_agg_df.date<='2023-06-05']
Q2_test_set_df = q2_agg_df[q2_agg_df.date>='2023-06-05']

In [ ]:
Q2_sales_data_df.head()

In [ ]:
changepoint_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()
seasonality_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()

In [ ]:
# getting the best parametes for Question 2
params_dict2 = best_params(Q2_sales_data_df)

In [ ]:
params_dict2

In [ ]:
prediction_days = 16
forecast_start_date2 = max(Q2_sales_data_df.ds) - timedelta(prediction_days)

category_df2 = Q2_sales_data_df[['ds','y']]


#PROPHET MODEL

#model
m2 = Prophet(changepoint_prior_scale=params_dict2['changepoint_prior_scale'],
            seasonality_prior_scale=params_dict2['seasonality_prior_scale'],
            holidays=holiday)

m2.fit(category_df2)

future2 = m2.make_future_dataframe(periods=78, freq='W-SAT')
fcst_prophet_train2 = m2.predict(future2)

filter = fcst_prophet_train2['ds']>=forecast_start_date2
predicted_df2 = fcst_prophet_train2[filter][['ds','yhat']]
predicted_df2 = predicted_df2.merge(category_df2)

print(mape(predicted_df2['y'],predicted_df2['yhat']))

In [ ]:
print(forecast_start_date2)

In [ ]:
fig12 = m2.plot(fcst_prophet_train2)
fig22 = m2.plot_components(fcst_prophet_train2)

forecasted_df2 = fcst_prophet_train2[fcst_prophet_train2['ds']>=forecast_start_date2]

In [ ]:
forecast_start_date2 = '2023-06-05'

future2 = m2.make_future_dataframe(periods=16, freq='W-SAT')
fcst_prophet_train2 = m2.predict(future2)

filter = fcst_prophet_train2['ds']>=forecast_start_date2
predicted_df2 = fcst_prophet_train2[filter][['ds','yhat']]



# Calculate Mean Absolute Percentage Error (MAPE)
mape_2 = mape(Q2_test_set_df['unit_sales'],predicted_df2['yhat'])


# Calculate Mean Absolute Error (MAE)
mae2 = mean_absolute_error(Q2_test_set_df['unit_sales'],predicted_df2['yhat'])

# Calculate Mean Squared Error (MSE)
mse2 = mean_squared_error(Q2_test_set_df['unit_sales'],predicted_df2['yhat'])

# Calculate Root Mean Squared Error (RMSE)
rmse2 = np.sqrt(mse2)

# Calculate R-squared (R2)
r2_2 = r2_score(Q2_test_set_df['unit_sales'],predicted_df2['yhat'])

print(f"-----------------------Features---------------------------")

print(f'Mean Absolute Error- MAE: {mae2}')
print(f'Mean Squared Error- MSE: {mse2}')
print(f'Root Mean Square Error- RMSE: {rmse2}')
print(f'R2:R-squared{r2_2}')
print(f'Mean Absolute Percentage Error- MAPE: {mape_2}')

In [ ]:
forecast_start_date2_1 = "2024-03-16"

future2_1 = m2.make_future_dataframe(periods=60, freq='W-SAT')
fcst_prophet_train2_1 = m2.predict(future2_1)

filter = fcst_prophet_train2_1['ds']>=forecast_start_date2_1
predicted_df2_1 = fcst_prophet_train2_1[filter][['ds','yhat']]

In [ ]:
predicted_df2_1.head(20)

In [ ]:
# @title Predicted Value over Time

sns.lineplot(data=predicted_df2_1, x='ds', y='yhat')

From the above forecast we can see that if the drink is released 2 weeks before ester the sales will be around 6700 units and continues to grow and fall following the pattern above and can be expected to reach the high demand around 9000 in the 10th week or can be discontinued in the 8th week when the demand reaches the lowest at 6300 after first peak.

## <a name = "Q2"> </a> 5.3. Question 3

Writing the segments for the Qestion 3.  
a. caloric Segment - Diet  
b. Market Category - Energy   
c. Manufacturer - Swire-CC  
d. Brand - Venomous Blast  
e. Package Type - 16 Liquid Small  
f. Flavor - Kiwano

In [ ]:
q3_f1 = fact_demand_state_data['caloric_segment'] == 'DIET/LIGHT'
q3_f2 = fact_demand_state_data['category'] == 'ENERGY'
q3_f3 = fact_demand_state_data['manufacturer'].str.lower().str.contains('swire')
q3_f4 = fact_demand_state_data['item'].str.lower().str.contains('kiwano')
q3_f5 = fact_demand_state_data['brand'].str.lower().str.contains('venomous blast')

filter_condition_q3 = ( q3_f1 & q3_f2 & q3_f3 & q3_f4 & q3_f5 )

q3_df = fact_demand_state_data[filter_condition_q3]

q3_df.head()

In [ ]:
q3_df.shape

In [ ]:
min(q3_df['date']), max(q3_df['date'])

In [ ]:
#q2_agg_df = q2_df.groupby(['date']).agg({'unit_sales':'sum'}).reset_index().sort_values(['date'])
q3_agg_df = q3_df.groupby(['date']).agg({'unit_sales': 'sum', 'item': 'nunique'}).reset_index().sort_values(['date'])
q3_agg_df['unit_sales'] = q3_agg_df['unit_sales'] / q3_agg_df['item']
q3_agg_df.drop(columns=['item'],inplace=True)

In [ ]:
q3_agg_df.head()

In [ ]:
# @title date vs sum of unit_sales for Q2

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
    # Aggregate sum of unit sales by date
  summed = series.groupby('date')['unit_sales'].sum().reset_index()

  xs = summed['date']
  ys = summed['unit_sales']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

# Example usage:
fig, ax = plt.subplots(figsize=(10, 5.2))
df_sorted = q3_agg_df.sort_values('date', ascending=True)
_plot_series(df_sorted, 'Sum of Unit Sales')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
plt.ylabel('Sum of Unit Sales')
plt.legend()
plt.show()

In [ ]:
Q3_sales_data_df = q3_agg_df[q2_agg_df.date<='2023-06-05']
Q3_test_set_df = q3_agg_df[q2_agg_df.date>='2023-06-05']

In [ ]:
Q3_sales_data_df.head()

In [ ]:
changepoint_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()
seasonality_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()

In [ ]:
# getting the best parametes for Question 2
params3_dict = best_params(Q3_sales_data_df)

In [ ]:
params3_dict

In [ ]:
prediction_days = 16
forecast_start_date3 = max(Q3_sales_data_df.ds) - timedelta(prediction_days)

category_df3 = Q3_sales_data_df[['ds','y']]


#PROPHET MODEL

#model
m3 = Prophet(changepoint_prior_scale=params3_dict['changepoint_prior_scale'],
            seasonality_prior_scale=params3_dict['seasonality_prior_scale'],
            holidays=holiday)

m3.fit(category_df3)

future3 = m3.make_future_dataframe(periods=78, freq='W-SAT')
fcst_prophet_train3 = m3.predict(future3)

filter = fcst_prophet_train3['ds']>=forecast_start_date3
predicted_df3 = fcst_prophet_train3[filter][['ds','yhat']]
predicted_df3 = predicted_df3.merge(category_df3)

print(mape(predicted_df3['y'],predicted_df3['yhat']))

In [ ]:
print(forecast_start_date3)

In [ ]:
fig12 = m3.plot(fcst_prophet_train3)
fig22 = m3.plot_components(fcst_prophet_train3)

forecasted_df3 = fcst_prophet_train3[fcst_prophet_train3['ds']>=forecast_start_date3]

In [ ]:
Q3_test_set_df.head()

In [ ]:
Q3_test_set_df.shape

In [ ]:
forecast_start_date3 = '2023-07-08'

future3 = m3.make_future_dataframe(periods=8, freq='W-SAT')
fcst_prophet_train3 = m3.predict(future3)

filter = fcst_prophet_train3['ds']>=forecast_start_date3
predicted_df3 = fcst_prophet_train3[filter][['ds','yhat']]



# Calculate Mean Absolute Percentage Error (MAPE)
mape_3 = mape(Q3_test_set_df['unit_sales'],predicted_df3['yhat'])


# Calculate Mean Absolute Error (MAE)
mae3 = mean_absolute_error(Q3_test_set_df['unit_sales'],predicted_df3['yhat'])

# Calculate Mean Squared Error (MSE)
mse3 = mean_squared_error(Q3_test_set_df['unit_sales'],predicted_df3['yhat'])

# Calculate Root Mean Squared Error (RMSE)
rmse3 = np.sqrt(mse3)

# Calculate R-squared (R2)
r2_3 = r2_score(Q3_test_set_df['unit_sales'],predicted_df3['yhat'])

print(f"-----------------------Features---------------------------")

print(f'Mean Absolute Error- MAE: {mae3}')
print(f'Mean Squared Error- MSE: {mse3}')
print(f'Root Mean Square Error- RMSE: {rmse3}')
print(f'R2:R-squared{r2_3}')
print(f'Mean Absolute Percentage Error- MAPE: {mape_3}')

# <a name = "Conclusion"> </a> 7. Conclusion

Overall, we found that the Prophet model is good as it provided the less MAPE value, which is the Percentage error for the Mean Absolute values. So when error is less that means that the model is good in predicting the

# <a name = "group"> </a> 8. Group Member Contribution

1. **Tom Kingston:**  


2. **Sushma Sree Mutyapu:**


3. **Sai Anogna Chittudi:** I have created the Prophet model for the Questions 1,2 and 3 along with Visulizations and hyperparameter tuning for each of the datasets seperately. Completed the Introduction and the data Preparation sections.

4. **Greg Francom:**